In [1]:
import os
import tensorflow as tf
import numpy as np

In [4]:
NUM_CLASSES = 11

In [5]:
def prepare_data(folder):
    
    # Creating subdirectories for 11 classes.
    for i in range(NUM_CLASSES):
        os.mkdir(path=folder + str(i))
        
    # Move the images to specific directories
    for path in os.listdir(folder):
        if os.path.isdir(folder + path) == False:
            file_array = path.split('_')
            os.rename(folder + path, folder + file_array[0] + '/' + file_array[1])

In [7]:
prepare_data('./evaluation/')

In [17]:
TARGET_DIM = 224
BATCH_SIZE = 32

In [18]:
def get_datagen(preprocessing_fun):
    return tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocessing_fun,
        rescale=1.0/255.0
    )

In [19]:
def evaluate():
    
    preprocessing_funs = [
        tf.keras.applications.mobilenet.preprocess_input,
        tf.keras.applications.resnet50.preprocess_input,
        tf.keras.applications.xception.preprocess_input
    ]
    
    predictions = []
    models_list = os.listdir('./models/')
    
    for i in range(len(models_list)):
        
        print("Evaluating " + str(models_list[i]))
        model = tf.keras.models.load_model('./models/' + models_list[i])
        datagen = get_datagen(preprocessing_funs[i])
        
        test_generator = datagen.flow_from_directory(
            directory='./evaluation/', 
            target_size=(TARGET_DIM, TARGET_DIM),
            batch_size=BATCH_SIZE,
            shuffle=False
        )
        
        predictions.append(model.evaluate_generator(test_generator, steps=test_generator.samples // BATCH_SIZE))
    
    return predictions

In [20]:
predictions = evaluate()

Evaluating mobilenet-41-0.74.hdf5
Found 3347 images belonging to 11 classes.
Instructions for updating:
Please use Model.evaluate, which supports generators.
Evaluating resnet50-47-0.77.hdf5
Found 3347 images belonging to 11 classes.
Evaluating xception-72-0.80.hdf5
Found 3347 images belonging to 11 classes.


In [22]:
predictions = np.array(predictions)

In [27]:
print("Maximum accuracy: " + str(max(predictions[:, 1])))

Maximum accuracy: 0.817307710647583


In [28]:
print("Model with max accuracy: " + str(np.argmax(predictions[:, 1])))

Model with max accuracy: 2


### Results:

Xception model yields maximum accuracy on test data with 0.8173